# WeRateDogs Twitter Data Wrangling

This project is to gather the data from [WeRateDogs (@dog_rates)](https://twitter.com/dog_rates), then assess and clean the data for further exploratory data analysis, and also training a recurrent neural network to identify the species of dogs.

The project focuses on below targets:
1. Keep the original ranks from WeRateDogs only, not the ones from retweet 
2. At least 8 quality issues and 2 tidiness issues
3. Including merge data table to achieve tidiness target
4. Be noticed that numerator sometimes is larger than denominator in this dataset, and that is the way WeRateDogs runs

The process of this project is as below:
1. Data wrangling, including cleaning, assessing and cleaning in programmatic ways
2. Store, analyze and visualize the processed data
3. Summary the first two steps in a report

In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import re

## Data Gathering

There are three files to be gathered:
1. Download twitter data of WeRateDogs from [GitHub repo](https://github.com/udacity/new-dand-advanced-china/tree/master/数据清洗/WeRateDogs项目)
2. Download programmatically for the image recognition result of dog species, in above repo
3. Gather retweet_count and favorite_count via `Tweepy` API, and combine them into a JSON format txt file, each line representing a single record, at least including `tweet ID`, `retweet_count` and `favorite_count` information 

After gathering, import all into separated `pandas.DataFrame` 

In [2]:
##### 1. Import existed dataset, including tweet ID #####
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [4]:
##### 2. Programmatically download image recoginition result ######
image_predictions = pd.read_csv('https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv',
                               sep='\t')
image_predictions.to_csv('image-predictions.tsv')
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [ ]:
##### 3. Info from Twitter API via tweepy #####
### Create tweepy instance for Twitter API ###
# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_token_secret = ''

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
### Looping to download all info and store in a list instance ###
# tweet_json = []
# tweet_error = {}
# for tweet_id in twitter_archive_enhanced.tweet_id:
#     try:
#         tweet = api.get_status(tweet_id, tweet_mode='extended')
#         tweet_json.append(tweet._json)
#     except Exception as e:
#         tweet_error[tweet_id] = e

In [38]:
### Re-download the error part ###
# tweet_error_2nd = {}
# for tweet_id in tweet_error.keys():
#     try:
#         tweet = api.get_status(tweet_id, tweet_mode='extended')
#         tweet_json.append(tweet._json)
#     except Exception as e:
#         tweet_error_2nd[tweet_id] = e

Rate limit reached. Sleeping for: 343


In [41]:
# tweet_error_2nd

{704761120771465216: tweepy.error.TweepError("Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)"),
 775096608509886464: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 802247111496568832: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 827228250799742977: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 837012587749474308: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 842892208864923648: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 845459076796616705: tweepy.error.TweepError([{'code': 144,
                           'message': 'No status found with that ID.'}]),
 861769973181624320: tweepy.error.

In [40]:
### From above, there are 11 tweet_id are unable to find corresponding info ###
### Re-download the last one ###
# tweet = api.get_status('704761120771465216', tweet_mode='extended')
# tweet_json.append(tweet._json)

In [46]:
### Store result in txt file ###
# with open('tweet_json_YY.txt', 'w') as output:  
#     for i in range(len(tweet_json)):
#         json.dump(tweet_json[i], output)

In [17]:
### Import tweet_json.txt to pandas.DataFrame ###
# Only include tweet ID, retweet_count and favorite_count #
json_str = []
with open('tweet_json.txt', encoding='utf-8') as json_file:
    for i in range(2352):   # there are 2352 rows in txt file got from visual inspection
        tweet = json_file.readline()
        json_str.append(tweet)

In [18]:
# Transform string read from txt to dic, and store in list to create DataFrame #
json_list = []
for i in range(len(json_str)):
    json_list.append(json.loads(json_str[i]))

tweet_json = pd.DataFrame(json_list)[['id', 'retweet_count', 'favorite_count']]

In [19]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
id                2352 non-null int64
retweet_count     2352 non-null int64
favorite_count    2352 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [19]:
# Export to csv for visual assessment in next step #
tweet_json.to_csv('tweet_json.csv')

### Summary of Data Collection

Corresponding to three targets,
1. Existed file was loaded via `pandas.read_csv` from local path, storing as `twitter_archive_enhanced` dataframe
2. Image recognition result was downloaded via `pandas.read_csv` from url, storing as `image_predictions` dataframe
3. Tweet info downloaded via tweepy was stored in `tweet_json_YY.txt`, to be distinguishable from the provided `tweet_json.txt` . However, to be eaiser for project reivew, `tweet_json.txt` was still used to generate `tweet_json` dataframe for the rest of project.

## Data Assessing

Accomplish at least 8 quality issues and 2 tidiness issues.

In [33]:
twitter_archive_enhanced.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
dtype: object

In [86]:
##### Check the max number of items in expanded_urls #####
urls_split = twitter_archive_enhanced_clean.expanded_urls.str.strip().str.split(',')

url_count = 1
missing_cell = []
for i in range(len(urls_split)):
    try:
        if len(urls_split[i])>url_count:
            url_count = len(urls_split[i])
    except Exception as e:
        missing_cell.append(i)
print ('Max number of items in a cell is ' + str(url_count))

Max number of items in a cell is 8


### Quality
#### Table `twitter_archive_enhanced` 
- column `rating_denominator` has cell not equaling to 10. Some are due to multiple dogs in the picture, and some are due to wrong extraction from text. *(Validity and Consistency)*
- column `name` has 55 values equaling to 'a', and another words like 'an', 'actually' etc, but actually there is no dog names in text *(Validity)*
- column `source` is recored as html element but the useful part is only the content *(Validity)*
- columns `timestamp` and `retweeted_status_timestamp` have '+0000' after all time but meaningless
- because we only consider the tweets with original rates and pictures, retweet and tweet without pictures should be removed
- columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` and `retweeted_status_user_id` should be int64 dtype
- columns `timestamp` and `retweeted_status_timestamp` should be datetime dtype
- all 'None' values should be set as missing values using NaN

#### Table `image_predicitons` 
- columns `p1`, `p2`, `p3` have cells starting with either upper case or lower case letter *(Consistency)*

### Tidiness
- (`twitter_archive_enhanced`) column `expanded_urls` has some cells with twitter urls and external urls, or duplicated urls
- (`twitter_archive_enhanced`) columns `doggo`, `floofer`, `pupper` and `puppo` is actually one variable
- table `tweet_json` and `image_predictions` are parts of `twitter_archive_enhanced` 

## Data Cleaning

Accomplish at least 8 quality issues and 2 tidiness issues.

In [39]:
##### Dummy read from file #####
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image-predictions.tsv', index_col=0)
tweet_json = pd.read_csv('tweet_json.csv', index_col=0)

In [40]:
##### Copy original data to be cleaned ######
twitter_archive_enhanced_clean = twitter_archive_enhanced.copy()
image_predictions_claen = image_predictions.copy()
tweet_json_clean = tweet_json.copy()

There is no completeness issues, so start with tidiness issues.

### Tidiness

#### 1. (twitter_archive_enhanced) column `expanded_urls` has some cells with twitter urls and external urls, or duplicated urls

##### Define 1
1. Split urls by comma, remove duplicated urls
2. Filter out twitter urls and outter urls into separated columns

##### Code 1

In [41]:
### Split out urls as items in list ###
urls_split = twitter_archive_enhanced_clean.expanded_urls.str.strip().str.split(',')

### Remove duplicated urls ###
missing_cell = []
for i in range(len(urls_split)):
    try:
        urls_split[i] = list(set(urls_split[i]))
    except Exception as e:
        missing_cell.append(i)
        
##### Check the max number of items in expanded_urls after cleaning #####
url_count = 1
missing_cell = []
for i in range(len(urls_split)):
    try:
        if len(urls_split[i])>url_count:
            url_count = len(urls_split[i])
    except Exception as e:
        missing_cell.append(i)
print ('Max number of items in a cell is ' + str(url_count))

##### Split out twitter url and outter url #####
regex = re.compile('https://twitter.*')
twitter_urls = []
outter_urls = []

for i in range(len(urls_split)):
    
    if urls_split[i] != urls_split[i]:
        twitter_urls.append(np.nan)
        outter_urls.append(np.nan)
    
    elif list(filter(regex.match, urls_split[i])):
        t_url = list(filter(regex.match, urls_split[i]))
        twitter_urls.append(t_url[0])
        outter_urls.append(np.nan)
    else:
        twitter_urls.append(np.nan)
        outter_urls.append(urls_split[i][0])

### Generate new columns and append on original dataframe ###
clean_url_dict = {'twitter_urls': twitter_urls,
                  'outter_urls': outter_urls}
clean_urls = pd.DataFrame(clean_url_dict)

### Drop original column and merge new columns to twitter_archive_enhanced_clean ###
twitter_archive_enhanced_clean.drop('expanded_urls', axis=1, inplace=True)
twitter_archive_enhanced_clean = pd.concat([twitter_archive_enhanced_clean, clean_urls], axis=1)

Max number of items in a cell is 2


##### Test 1

In [42]:
twitter_archive_enhanced_clean.twitter_urls.head(5)

0    https://twitter.com/dog_rates/status/892420643...
1    https://twitter.com/dog_rates/status/892177421...
2    https://twitter.com/dog_rates/status/891815181...
3    https://twitter.com/dog_rates/status/891689557...
4    https://twitter.com/dog_rates/status/891327558...
Name: twitter_urls, dtype: object

In [43]:
first_notnull = twitter_archive_enhanced_clean.outter_urls.first_valid_index()
twitter_archive_enhanced_clean.outter_urls.iloc[first_notnull-2:first_notnull+3]

308                              NaN
309                              NaN
310    https://vine.co/v/5W2Dg3XPX7a
311                              NaN
312                              NaN
Name: outter_urls, dtype: object

-----

#### 2. (twitter_archive_enhanced) columns doggo, floofer, pupper and puppo is actually one variable

##### Define 2

1. Combine four stage columns
2. Manual update cell with multiple stage info by understanding each tweet

##### Code 2

In [44]:
##### Combine four stage columns into one list #####
stage = twitter_archive_enhanced_clean[['doggo', 'floofer', 'pupper', 'puppo']]

dogs_list = []
for i in range(len(twitter_archive_enhanced_clean)):
    dogs = list(set(stage.iloc[i].values))
    if len(dogs)>1:
        dogs.remove('None')
    dogs_list.append(dogs)

In [45]:
##### Manual handle cell with double dogs #####
### Return a index list of cell with double dogs ###
multi_dogs = []
length = 1
for i in range(len(dogs_list)):
    if len(dogs_list[i])>1:
        multi_dogs.append(i)
        if len(dogs_list[i])>length:
            length = len(dogs_list[i])
print ('Maximum length of single cell: '+str(length))

Maximum length of single cell: 2


In [46]:
### Read tweets and understand their real meaning ###
dogs_list[multi_dogs[0]] = ['puppo']  # 191
dogs_list[multi_dogs[1]] = ['floofer']  # 200
dogs_list[multi_dogs[2]] = ['pupper']  # 460
dogs_list[multi_dogs[3]] = dogs_list[multi_dogs[3]]  # two dogs in pic 531
dogs_list[multi_dogs[4]] = dogs_list[multi_dogs[4]]  # two dogs in pic 565
dogs_list[multi_dogs[5]] = ['pupper']  # 575
dogs_list[multi_dogs[6]] = ['doggo']  # 705
dogs_list[multi_dogs[7]] = dogs_list[multi_dogs[7]]  # two dogs in pic 733
dogs_list[multi_dogs[8]] = dogs_list[multi_dogs[8]]  # two dogs in pic 778 RT
dogs_list[multi_dogs[9]] = dogs_list[multi_dogs[9]]  # two dogs in pic 822 RT
dogs_list[multi_dogs[10]] = dogs_list[multi_dogs[10]]  # two dogs in pic 889
dogs_list[multi_dogs[11]] = ['None']  # 956
dogs_list[multi_dogs[12]] = dogs_list[multi_dogs[12]]  # two dogs in pic 1063
dogs_list[multi_dogs[13]] = dogs_list[multi_dogs[13]]  # two dogs in pic 1113

In [47]:
##### Create new columns and update original DataFrame ######
dogs_dict = {'dog_stage': list(",".join(map(str,i)) for i in dogs_list)}
dog_stage = pd.DataFrame(dogs_dict)
twitter_archive_enhanced_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)
twitter_archive_enhanced_clean = pd.concat([twitter_archive_enhanced_clean, dog_stage], axis=1)

##### Test 2

In [48]:
twitter_archive_enhanced_clean.dog_stage.iloc[10:16]

10     None
11     None
12    puppo
13     None
14    puppo
15     None
Name: dog_stage, dtype: object

-----

#### 3. table `tweet_json` and `image_predictions` are parts of `twitter_archive_enhanced`

##### Define 3

1. Join `retweet_count` and `favorite_count` from `tweet_json` to `twitter_archive_enhanced`
2. Join dog species with highest probability of prediciton from `image_predicitons` to `twitter_archive_enhanced`

##### Code 3

In [49]:
##### Join with tweet_json #####
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean,
                                          tweet_json,
                                          left_on='tweet_id', right_on='id',
                                          how='left')
twitter_archive_enhanced_clean.drop(['id'], axis=1, inplace=True)

In [62]:
##### Join with image_prediction #####
### Select out highest prob dog prediction ###
tweet_id = []
species_list = []
species_prob_list = []

for i in range(len(image_predictions)):
    
    tweet_id.append(image_predictions.tweet_id[i])
    
    if image_predictions.p1_dog[i]==True:
        species_list.append(image_predictions.p1[i])
        species_prob_list.append(image_predictions.p1_conf[i])
    
    elif image_predictions.p2_dog[i]==True:
        species_list.append(image_predictions.p2[i])
        species_prob_list.append(image_predictions.p2_conf[i])
    
    elif image_predictions.p3_dog[i]==True:
        species_list.append(image_predictions.p3[i])
        species_prob_list.append(image_predictions.p3_conf[i])
    
    else:
        species_list.append(np.nan)
        species_prob_list.append(np.nan)
        
### Create DataFrame and join ###
image_pred_dict = {'tweet_id': tweet_id,
                   'species': species_list,
                   'species_prob': species_prob_list}
image_pred = pd.DataFrame(image_pred_dict)
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean,
                                          image_pred,
                                          left_on='tweet_id', right_on='tweet_id',
                                          how='left')

##### Test 3

In [68]:
twitter_archive_enhanced_clean.species.head(5)

0                   NaN
1             Chihuahua
2             Chihuahua
3    Labrador_retriever
4                basset
Name: species, dtype: object

In [69]:
twitter_archive_enhanced_clean.species_prob.head(5)

0         NaN
1    0.323581
2    0.716012
3    0.168086
4    0.555712
Name: species_prob, dtype: float64

## Data Storage and Visualization

1. Store the processed data as `twitter_archive_master.csv`
2. Illustrate at least 3 intuitions from data analysis and 1 visualization

## Sumamry Report

1. 300-to-600-word report for internal assessment to complete project, about how the whole project was done, saved as `wrangle_report.pdf` 
2. ~250-word report for external demonstration like blog post, saved as `act_report.pdf` 